## Check for data collection and processing errors

### Kingfisher Collect Log

Print the crawler statistics from the log file specified in the setup section. If `downloader/response_status_count/{code}` is non-zero and `{code}` is an HTTP error code (400-599), then the collection may be incomplete. Where possible, you should check the total number of releases and/or contracting processes against the front-end of the data source.

In [ ]:
if log_url != '':

  response = requests.get(log_url, auth=('scrape', scrapy_password))

  with open('log_file', 'wb') as f:
    f.write(response.content)
  
  log = ScrapyLogFile('log_file').logparser
  pprint(dict(log['crawler_stats']))

Generate a list of notes for each collection from the `collection_note` table.
Users can add notes when starting a spider and when running the `load` command. Processing steps also add notes at different levels.

- INFO:
  - load: The Kingfisher Collect crawl's statistics and reason for closing.
- WARNING:
  - compile: If a record has undated releases, linked releases or no releases, but a compiled release can be calculated (by using any dated releases, the `compiledRelease` field, or an undated release with a 'compiled' tag).
- ERROR:
  - load: If the input data is invalid JSON.
  - load: If Kingfisher Collect yielded a `FileError` item.
  - compile: If no compiled release can be calculated for a record.

In [ ]:
%%sql

SELECT
    collection_id,
    code,
    note,
    data
FROM
    collection_note
WHERE
    collection_id IN :collection_ids
